## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## preparation

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import pickle
import cv2
import glob

%matplotlib inline

# Make this mode to False, so it will only process static images with verbose image output
# if true, the static images will be processed still, but no interim images drawn
# when submit this work, it should be set to True
VIDEO_MODE = True

# plot input vs output side by side
# note that imshow expects rgb img

def plot_two_imgs(left_img, right_img, left_cmap=None, right_cmap=None, left_title = "left img", right_title= "right img", ):
    if VIDEO_MODE:
        return
    
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(left_img, cmap=left_cmap)
    ax1.set_title(left_title, fontsize=50)
    ax2.imshow(right_img, cmap=right_cmap)
    ax2.set_title(right_title, fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)



## First, I'll compute the camera calibration using chessboard images

In [2]:
# calibrate camera using collection of chessboard pictures. 
# Note: 
# 1. calibration will not happen if found coners less than half of the checkboard images 
# 2. Same image size for the correlation image and future image to be undistorted
# images_path: input file name patterns, default value:camera_cal/calibration*.jpg
# corner_num_x: number of corners along the x axis, default value:9
# corner_num_y: number of corners along the y axis, default value:6
# output_path:  output the corner found images for debugging purpose. default value:"camera_cal_output"
# img_size: this corresponds to the picture to be undistorted. default value: None, use the calibration img size
#
# The size of the image, which is passed into the calibrateCamera function, is just the height and width of the image.
# One way to retrieve these values is by retrieving them from the grayscale image shape array gray.shape[::-1]. 
# This returns the image width and height in pixel values like (1280, 960).
# Another way to retrieve the image shape, is to get them directly from the color image by retrieving the first two 
# values in the color image shape array using img.shape[1::-1]. This code snippet asks for just the first two values 
# in the shape array, and reverses them. Note that in our case we are working with a greyscale image, 
# so we only have 2 dimensions (color images have three, height, width, and depth), so this is not necessary.

# Return value - same as cv2.calibrateCamera return
# plus result saved to <output_path>/cam_calibration_result_pickle.p for future use
 

def calibrate_camera (images_path="camera_cal/calibration*.jpg", corner_num_x=9, corner_num_y=6, output_path="camera_cal_output/", img_size=None):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((corner_num_y*corner_num_x,3), np.float32)
    objp[:,:2] = np.mgrid[0:corner_num_x, 0:corner_num_y].T.reshape(-1,2)

    # Make a list of calibration images
    images = glob.glob(images_path)


    objpoints = [] # 3D points in real world space
    imgpoints = [] # 2D points in img plane

    # prepare output file dir
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Step through the list and search for chessboard corners
    corners_found = 0
    for idx, fname in enumerate(images):
        img = mpimg.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (corner_num_x,corner_num_y), None)

        # If found, add object points, image points
        if ret == True:
            corners_found += 1
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            cv2.drawChessboardCorners(img, (corner_num_x,corner_num_y), corners, ret)
            write_name = output_path+'corners_found_in_'+os.path.basename(fname)
            mpimg.imsave(write_name, img)
            #print ("corners found and saved as:" + write_name)

        else:
            print ("coners not found in: " + fname +" skip it")
       
    
    if corners_found > idx / 2:
        print ("OK: found cornders in more than half of the input images, carry on with calibration")
        if img_size is None:
            img_size = (img.shape[1], img.shape[0])
            print("img_size is None, use calibration img size: " + str(img_size))
            
            
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
        
        # Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
        dist_pickle = {}
        dist_pickle["mtx"] = mtx
        dist_pickle["dist"] = dist
        cam_calibration_result = output_path+"cam_calibration_result_pickle.p"
        pickle.dump( dist_pickle, open( cam_calibration_result, "wb" ) )
        
        print("camera calibration result saved in file " + cam_calibration_result + " for future use" )
    
        return ret, mtx, dist, rvecs, tvecs
        
    else:
        print ("Error: found cornders in less than half of the input images, stopped calibration.")
        return -1, None, None, None, None



##  Note of Applying a distortion correction to raw images.
No wrapping code needed. please use cv2.undistort directly in the future pipeline.
Example code below: 

In [3]:
# Read in the saved camera matrix and distortion coefficients from the pickle dump file
# dist_pickle = pickle.load( open( "wide_dist_pickle.p", "rb" ) )
# mtx = dist_pickle["mtx"]
# dist = dist_pickle["dist"]
# Use the OpenCV undistort() function to remove distortion
# undist = cv2.undistort(img, mtx, dist, None, mtx)

In [4]:
## some tools before proceed, including Gaussian blur and use region of interest so we can skip other noise may occur

In [5]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image



## Then, Use color transforms, gradients, etc., to create a thresholded binary image

In [6]:
# Define a function that takes an image, gradient orientation,
# and threshold min / max values.

# the gray image color space has bad result with shadows, try 

def abs_sobel_thresh(gray_img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray_img, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray_img, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    grad_binary = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Return the result
    return grad_binary


# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(gray_img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray_img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    dir_binary =  np.zeros_like(absgraddir)
    dir_binary[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return dir_binary

# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(gray_img, sobel_kernel=3, thresh=(0, 255)):

    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray_img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= thresh[0]) & (gradmag <= thresh[1])] = 1
    

    # Return the binary image
    return mag_binary

# in HLS color space, the the S channel is still doing a fairly robust job of picking up the lines 
# under very different color and contrast conditions, which helps to  Reliably detect different colors 
# of lane lines under varying degrees of daylight and shadow.
def color_thresh(hls_img, s_thresh=(170, 255)):
    s_channel = hls_img[:,:,2]
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    return s_binary

# a larger kernel size make it smoother
def combined_thresh(rgb_img, ksize=3, grad_thresh=(0, 255), magitude_thresh=(0, 255), dir_thresh=(0, np.pi/2), s_threash=(170,255)):
    
    # for plotting purpose.
    hls_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HLS)


    # as input to various threshold 
    gray_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2GRAY)
    
    # Define a kernel size for Gaussian smoothing / blurring
    # Kernel Must be an odd number (3, 5, 7...)
    blurgray_img = gaussian_blur(gray_img, 3) 

    
    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(blurgray_img, orient='x', sobel_kernel=ksize, thresh=grad_thresh)    
    #plot_two_imgs(left_img=gray_img, left_cmap='gray', right_img=gradx, right_cmap='gray', right_title = "gradx" )
        
    grady = abs_sobel_thresh(blurgray_img, orient='y', sobel_kernel=ksize, thresh=grad_thresh)
    #plot_two_imgs(left_img=gray_img, left_cmap='gray', right_img=grady, right_cmap='gray', right_title = "grady" )

    mag_binary = mag_thresh(blurgray_img, sobel_kernel=ksize, thresh=magitude_thresh)  
    #plot_two_imgs(left_img=gray_img, left_cmap='gray', right_img=mag_binary, right_cmap='gray', right_title = "mag" )

    dir_binary = dir_threshold(blurgray_img, sobel_kernel=ksize, thresh=dir_thresh)
    #plot_two_imgs(left_img=gray_img, left_cmap='gray', right_img=dir_binary, right_cmap='gray', right_title = "directional" )

    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    #plot_two_imgs(left_img=gray_img, left_cmap='gray', right_img=combined, right_cmap='gray', right_title = "combined" )


    # apply color
    s_binary = color_thresh(hls_img, s_threash)
    #plot_two_imgs(left_img=hls_img, right_img=s_binary, right_title = "combined" )
    
    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    # stacked_color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

    # Combine the two binary thresholds
    combined_with_color = np.zeros_like(combined)
    combined_with_color[(s_binary == 1) | (combined == 1)] = 1
    plot_two_imgs(left_img=rgb_img, right_img=combined_with_color, right_cmap='gray', right_title = "combined" )


    return combined_with_color


## Transform perspective 
Use one of the test image for perspective transform and then use it for other images assuming the perspective didn't change in this project.

In [7]:
# Use "straight_lines1.jpg" as the sample
# example of using this matrix to undistort:
# img_size = (undistorted.shape[1], undistorted.shape[0])
# warped = cv2.warpPerspective(undistorted, M, img_size, flags=cv2.INTER_LINEAR)


def get_perspective_transform_matrix():
     
    # a) define 4 source points src = np.float32([[,],[,],[,],[,]])
    
    # Note, need to box in areas that you want to draw lane-zone in the future
    # this is to ensure we included enough lane segments
    # src = np.float32([[824,540],[1042,678],[262,678],[464,540]])
    #src = np.float32([[734,480],[1042,678],[262,678],[556,480]])
    src = np.float32([[748,490],[1050,678],[262,678],[540,490]])



    #Note: you could pick any four of the detected corners 
    # as long as those four corners define a rectangle
    #One especially smart way to do this would be to use four well-chosen
    # corners that were automatically detected during the undistortion steps
    #We recommend using the automatic detection of corners in your code
    
    # b) define 4 destination points dst = np.float32([[,],[,],[,],[,]])
    #dst = np.float32([[1042,120],[1042,719],[262,719],[282,120]])
    dst = np.float32([[1042,5],[1042,719],[262,719],[282,5]])

    
    # above dest box is ok but not great, because it zoomed into a small area of the map only
    # zoom out using below ratio - no change to box center 
    #zoom_by = 0.3
    #dx = (dst[0][0]-dst[3][0])*zoom_by
    #dy = (dst[2][1] - dst[0][1])*zoom_by

    #new_dst = np.zeros_like(dst)
    # top-right, bottom-right, bottom-left, top-left
    #new_dst = [ [dst[0][0]-dx , dst[0][1]+dy], 
    #            [dst[1][0]-dx , dst[1][1]-dy], 
    #           [dst[2][0]+dx , dst[2][1]-dy],
    #            [dst[3][0]+dx , dst[3][1]+dy] ]

    # c) use cv2.getPerspectiveTransform() to get M, the transform matrix

    M = cv2.getPerspectiveTransform(src, dst)
    INVM = cv2.getPerspectiveTransform(dst, src)
                       
            
    # Return the resulting image and matrix
    return M, INVM




## Split the histogram for the two lines
The first step we'll take is to split the histogram into two sides, one for each lane line.

Set up windows and window hyperparameters
Our next step is to set a few hyperparameters related to our sliding windows, and set them up to iterate across the binary activations in the image. We have some base hyperparameters below, but don't forget to try out different values in your own implementation to see what works best!

Iterate through nwindows to track curvature
Now that we've set up what the windows look like and have a starting point, we'll want to loop for nwindows, with the given window sliding left or right if it finds the mean position of activated pixels within the window to have shifted.


In [8]:
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img








## Fit a polynomial
Now that we have found all our pixels belonging to each line through the sliding window method, it's time to fit a polynomial to the line. First, we have a couple small steps to ready our pixels.

In [9]:


# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = []  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float')
        # total number of matched pixels
        self.px_count = 0
        
        # hypger parameter for curvature and center position evaluation
        self.ym_per_pix = None  
        self.xm_per_pix = None 

    #Note, all_x and all_y stands for the pixels matches the lane,
    # but we don't store the raw info in order to make the Line object lean. 
    # instead, we derive all the needed information from here, e.g. count, curvature, centerpos, min-y, max-y
    
    def add_fit(self, fit, all_x, all_y):
        # add a found fit to the line, up to n
        if fit is not None:
            if self.best_fit is not None:
                # if we have a best fit, see how this new fit compares
                self.diffs = abs(fit-self.best_fit)
            if (self.diffs[0] > 0.001 or \
               self.diffs[1] > 1.0 or \
               self.diffs[2] > 100.) and \
               len(self.current_fit) > 0:
                self.detected = False
            else:
                self.detected = True
                self.px_count = np.count_nonzero(all_x)
                self.current_fit.append(fit)
                if len(self.current_fit) > 50:
                    # throw out old fits, keep newest n
                    self.current_fit = self.current_fit[len(self.current_fit)-50:]
                self.best_fit = np.average(self.current_fit, axis=0)
                
                # update the cuvrate etc. based on the best fit
                self.radius_of_curvature = self.measure_curvature(all_x, all_y)
                
        # or remove one from the history, if not found
        else:
            self.detected = False
            if len(self.current_fit) > 0:
                # throw out oldest fit
                self.current_fit = self.current_fit[:len(self.current_fit)-1]
            if len(self.current_fit) > 0:
                # if there are still any fits in the queue, best_fit is their average
                self.best_fit = np.average(self.current_fit, axis=0)
                
    # input:
    # img - binary img 
    # 

    def measure_curvature(self, allx, ally):
        '''
        Calculates the curvature of polynomial functions in meters.
        '''

        # Define y-value where we want radius of curvature
        # We'll choose the maximum y-value, corresponding to the bottom of the image
        y_eval = np.max(ally)

        # Fit a second order polynomial to pixel positions in each lane line
        # Fit new polynomials to x,y in world space
        fit_cr = np.polyfit(ally*self.ym_per_pix, allx*self.xm_per_pix, 2)

        # Calculation of R_curve (radius of curvature)
        curverad = ((1 + (2*fit_cr[0]*y_eval*self.ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
        
        return curverad



# Define a class to store both left and right line
class Lane():
    def __init__(self):
        
        # hypger parameter for curvature and center position evaluation
        self.ym = 30 # meters per pixel in y dimension, (weakly) assuming 30m length 
        self.xm = 3.7 # meters per pixel in x dimension, (strongly) assuming 3.7m width 
        self.img_shape = (720, 1280) # used to calcuate center position.
        self.warped_shape = (720, 1280) # used to represent the size of the bird eye view shape

        # each lane musg have a left line and a right line
        self.left_line = Line()
        self.left_line.ym_per_pix = self.ym / self.img_shape[0] 
        self.left_line.xm_per_pix = self.xm / self.img_shape[1] 
        
        self.right_line = Line()
        self.right_line.ym_per_pix = self.ym / self.img_shape[0] 
        self.right_line.xm_per_pix = self.xm / self.img_shape[1]


    #assume the camera is mounted at the center of the car and the deviation of the midpoint of the lane 
    # from the center of the image is the offset.
    def get_center_position(self):
        mypos = self.img_shape[1]/2
        
        h = self.img_shape[0]
        
        left_fit = self.left_line.best_fit
        right_fit = self.right_line.best_fit

        lfit = left_fit[0]*h**2 + left_fit[1]*h + left_fit[2]
        rfit = right_fit[0]*h**2 + right_fit[1]*h + right_fit[2]
        lpos = (rfit + lfit) /2
        center_dist = (mypos - lpos) * (self.xm / self.img_shape[1])
        
        return center_dist
    
    # lane zone shape is determined by the polyfit result. 
    # original_img is the orginal image as the backdrop
    # warped_img - is the one used to do polyfitting. used to determine shape info
    # Minv - perspective transform matrix , from warped bird eye view to unwarped camera view
    # left_line - detected result (note this is not orignal image pixel, but calcuated per the polyfit
    # right_line - deteed result
    
    def draw(self, original_img, Minv):
        # Create an image to draw the lines on
        warp_zero = np.zeros(self.warped_shape).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
        
        # Recast the x and y points into usable format for cv2.fillPoly()
        
        ## Generate x and y values for plotting
        ploty = np.linspace(0, self.warped_shape[0]-1, self.warped_shape[0] )
        
        left_fit = self.left_line.best_fit
        right_fit = self.right_line.best_fit
        
        try:
            left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
            right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        except TypeError:
            # Avoids an error if `left` and `right_fit` are still none or incorrect
            print('The function failed to fit a line!')
            left_fitx = 1*ploty**2 + 1*ploty
            right_fitx = 1*ploty**2 + 1*ploty
        
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
        
        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, Minv, (original_img.shape[1], original_img.shape[0])) 
        
            
        # Combine the result with the original image
        result = cv2.addWeighted(original_img, 1, newwarp, 0.3, 0)


        return result
        

In [10]:
# binary_warped - input binary image in warped bird-eye view
# output:
# ouptut_img - 
# left_line, right line in Line class

def fit_polynomial(binary_warped):
    
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    
    # prepare the result
    lane = Lane()
    
    lane.left_line.add_fit(left_fit, leftx, lefty)
    lane.right_line.add_fit(right_fit, rightx, righty)


    return out_img, lane

## Pipeline linking all processors

In [11]:
# detect lane from a image file
#


def adv_lane_detection_pipeline(rgb_img, calibration_mtx, calibration_dist):

    img_size = (rgb_img.shape[1], rgb_img.shape[0])
    
    # Apply a distortion correction to raw imageg
    undist = cv2.undistort(rgb_img, calibration_mtx, calibration_dist, None, calibration_mtx)
    

    # Use color transforms, gradients, etc., to create a thresholded binary image.
    # experiment about the grad and magnitude threshold:
    # lower the L-threshold, more noise. should not be lower than 30, 50 looks good.
    # higher the H-threshold, brighter the line. nothing wrong with it. use 200. 
    
    thresholded = combined_thresh(undist, 15, (50,200),(50,200),(0.7,1.3),(170,255))
    
    imshape = thresholded.shape
    
    #a typical lane zone - straightline [[748,490],[1050,678],[262,678],[540,490]]
    
    #vertices = np.array([[(0,imshape[0]),(430, 330), (620, 330), (imshape[1],imshape[0])]], dtype=np.int32)
    mgin = 150
    vertices = np.array([[(748+mgin,490-mgin),(1050+mgin,678+mgin), (262-mgin,678+mgin), (540-mgin,490-mgin)]], dtype=np.int32)
    masked_thresholded = region_of_interest(thresholded, vertices)


        
    # Apply a perspective transform to rectify binary image ("birds-eye view").
    M, INVM = get_perspective_transform_matrix()
    
    img_size = (masked_thresholded.shape[1], masked_thresholded.shape[0])
    warped = cv2.warpPerspective(masked_thresholded, M, img_size, flags=cv2.INTER_LINEAR)

    #plot_two_imgs(left_img=thresholded, left_cmap='gray', right_img=warped, right_cmap='gray', right_title = "warped" )
    
    
    # Detect lane pixels and fit to find the lane boundary.    
    lane_line_img, lane = fit_polynomial (warped) 
    plot_two_imgs(left_img=rgb_img, right_img=lane_line_img, right_title = "warped" )
    
    left_curv = lane.left_line.radius_of_curvature
    right_curv = lane.right_line.radius_of_curvature
    
    cpos = lane.get_center_position()
    
   
    # Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

    result = lane.draw(rgb_img, INVM)

    text1 = 'Curveature: ' + '{:04.1f}'.format(left_curv) + 'm'
    cv2.putText(result, text1, (60,50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (102,0,51), 2, cv2.LINE_AA)

    text2 = 'Center Offset: ' + '{:04.2f}'.format(cpos) + 'm' 
    cv2.putText(result, text2, (60,90), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (102,0,51), 2, cv2.LINE_AA)

    plot_two_imgs(left_img=rgb_img, right_img=result,left_title = "original", right_title = "lane highlighted" )


    return result




In [12]:
# Calibrate camera

ret, mtx, dist, rvecs, tvecs = calibrate_camera()

if ret < 0:
    print ("Error: calibration camera failed.")
    exit 

# iterate through every input file
INPUT_DIR = "test_images/"
OUTPUT_DIR = "test_images_output/"
os.makedirs(os.path.dirname(OUTPUT_DIR), exist_ok=True)

for filename in os.listdir(INPUT_DIR):  
    rgb_img = mpimg.imread(INPUT_DIR+filename)
    result_img = adv_lane_detection_pipeline (rgb_img, mtx, dist)
    mpimg.imsave(OUTPUT_DIR+filename, result_img)
    print("Processed img " + filename )
    
    # debug on, please remove below line, only do one img
    #break

coners not found in: camera_cal/calibration5.jpg skip it
coners not found in: camera_cal/calibration4.jpg skip it
coners not found in: camera_cal/calibration1.jpg skip it
OK: found cornders in more than half of the input images, carry on with calibration
img_size is None, use calibration img size: (1280, 720)
camera calibration result saved in file camera_cal_output/cam_calibration_result_pickle.p for future use
Processed img test6.jpg
Processed img test5.jpg
Processed img test4.jpg
Processed img test1.jpg
Processed img test3.jpg
Processed img test2.jpg
Processed img straight_lines2.jpg
Processed img straight_lines1.jpg


In [13]:
## Test on Video

In [14]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [15]:
def process_video_frame(input_img):
        
    output_img = adv_lane_detection_pipeline (input_img, mtx, dist)
    
    return output_img

In [16]:
def process_video_clip(input_video_name):

    video_input_dir = './'
    video_output_dir = 'test_videos_output/'
    os.makedirs(os.path.dirname(video_output_dir), exist_ok=True)

    clip1 = VideoFileClip(video_input_dir+input_video_name)
    lane_clip1 = clip1.fl_image(process_video_frame) #NOTE: this function expects color images!!
    %time lane_clip1.write_videofile(video_output_dir + input_video_name, audio=False)


if VIDEO_MODE:
    process_video_clip('project_video.mp4')
    process_video_clip('challenge_video.mp4')
    #process_video_clip('harder_challenge_video.mp4')
    
    

[MoviePy] >>>> Building video test_videos_output/project_video.mp4
[MoviePy] Writing video test_videos_output/project_video.mp4


100%|█████████▉| 1260/1261 [04:28<00:00,  4.82it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/project_video.mp4 

CPU times: user 17min 6s, sys: 1min 5s, total: 18min 12s
Wall time: 4min 29s
[MoviePy] >>>> Building video test_videos_output/challenge_video.mp4
[MoviePy] Writing video test_videos_output/challenge_video.mp4


 28%|██▊       | 134/485 [00:27<01:05,  5.39it/s]

TypeError: expected non-empty vector for x